# Data cleaning and pre-processing
### by Jonathan Fischer and Sasha Yousefi

In [ ]:
# Load the necessary modules
from datascience import *
import numpy as np
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import scipy.stats as stats
plt.style.use('fivethirtyeight')
import pandas as pd
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from client.api.notebook import Notebook

## Normalization

### Raw data

In [ ]:
# Load data using pd.read_csv('file_name', sep = ' ')
# matrix should be 5000 genes x 20 samples

raw_data = pd.read_csv('https://raw.githubusercontent.com/ds-connectors/Data88-Genetics_and_Genomics/master/Lab02/data_norm.csv', sep = ' ')

Let's say the first 10 columns are samples from healthy patients and the last ten from patients with an illness. We can assume that samples from the same condition should be similar to one another. To start, we can see whether they have similar numbers of total reads.

In [ ]:
# First, take the sum of each column. np.sum(df) does this by default for a data frame df
total_counts = ?

# Make a barplot of the total number of reads in each sample for the healthy patients to investigate whether
# they have similar numbers of total reads.
# To the right of each line there is a #. Please fill in with what you think each line of the code does

bars = list(raw_data.columns)                    #  ?
y_pos = np.arange(len(bars))                     #  ?
col_list = ['Black']*10 + ['Red']*10             #  ?

plt.figure(figsize=(12,6))                       #  ?
plt.bar(y_pos, total_counts, color = col_list)   #  ?
plt.xticks(y_pos, bars)                          #  ?
plt.title('Without normalization')               #  ?
plt.xlabel('Sample')                             #  ?
plt.ylabel('Total reads')                        #  ? 
plt.show()                                       #  ?

Let's see the effect of not normalizing on our ability to characterize differences in the gene expression profiles of healthy and sick patients. Since I created these, I can tell you that the overall median FC should be around 0.

In [ ]:
# First, compute the mean of each gene within healthy and sick individuals
healthy_means_raw = np.mean(raw_data.iloc[:,0:10], axis = 1)
sick_means_raw = ?          # now repeat for the "sick" individuals in columns 11-20

In [ ]:
# Now that we have a single vector for each condition, make a histogram of the log2 fold changes between them
# Remember to add 1 before taking the log, e.g. np.log2(x+1)
# We'll eventually compare this to what we get after normalization

raw_fc = ?         # take the difference between sick_means_raw and healthy_means_raw after taking their logs

plt.hist(raw_fc, bins = ?, density = True)   # we make a histogram with plt.hist. Set bins = 100 to get a nice figure
plt.title('Without normalization') 
plt.xlabel('Log2 Fold Changes')
plt.ylabel('Density')
plt.show()

### Transcripts per Million

A common normalization is to divide by some scaling factor which is proportional to the total number of reads in a sample. This has fallen out of favor a bit, and we'll see why.

In [ ]:
# Normalize the data using the Transcripts per Million (TPM) approach. 
# That is, divide each column by its sum and multiply by one million
# In this case, the division operator works column-wise on the df, so try df / total_counts * 1e6

TPM_data = ?

In [ ]:
# Repeat the histogram by taking means and displaying the log2 FCs between conditions after TPM normalization
healthy_means_TPM = ?
sick_means_TPM = ?

# Now that we have a single vector for each condition, make a histogram of the log2 fold changes between them
# We'll eventually compare this to what we get after normalization
TPM_fc = ?

# make histogram
# here:

??

plt.title('After TPM normalization') 
plt.xlabel('Log2 Fold Changes')
plt.ylabel('Density')
plt.show()

### Median normalization

An alternative normalization is to use a scaling factor which is proportional to the median value in a sample. This avoids some of the pitfalls of TPM, though more sophisticated methods are usually better.

In [ ]:
# Normalize the data my dividing each column by its median and then multiplying by 1000
# to get the column medians, try df.median()
medians = ?
MN_data = ?

In [ ]:
# Repeat the histogram by taking means and displaying the log2 FCs between conditions after median normalization
healthy_means_MN = ?
sick_means_MN = ?

# Now that we have a single vector for each condition, make a histogram of the log2 fold changes between them
# We'll eventually compare this to what we get after normalization
MN_fc = ?

# make histogram
# here:

??

plt.title('After median normalization') 
plt.xlabel('Log2 Fold Changes')
plt.ylabel('Density')
plt.show()

## Outliers

In [ ]:
# Load outlier data. Here we're initally assuming columns 1-10 are condition 1 and 11-25 are condition 2
outlier_table = pd.read_csv('https://raw.githubusercontent.com/ds-connectors/Data88-Genetics_and_Genomics/master/Lab02/data_outlier.csv', sep = ' ')

In [ ]:
# Compute number of genes with log2 FC >= 1 b/w conditions and plot a histogram of the log2 FCs
cond1_means = ?
cond2_means = ?

outlier_fc = ?

# We can sum booleans, or True/False values, to count how many are True. 
DE_1 = np.sum(outlier_fc >= 1)

# Make histogram
# here: 
??

plt.title('Without outlier detection') 
plt.xlabel('Log2 Fold Changes')
plt.ylabel('Density')
plt.show()

In [ ]:
# Run PCA and put the top PCs into a data frame
x = np.log2(outlier_table+1)
x = StandardScaler().fit_transform(x.transpose())
pca = PCA(n_components=5)
principalComponents = pca.fit_transform(x)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['PC1', 'PC2', 'PC3', 'PC4', 'PC5'])

In [ ]:
# Plot results of PCA and look for outliers
col_list = ['Black']*10 + ['Red']*15
plt.scatter(principalDf['PC1'], principalDf['PC2'], color = col_list)
plt.xlabel('PC 1')
plt.ylabel('PC 2')
plt.show()

In [ ]:
# Remove outliers. Now how many genes have abs log2 FC >= 1?
to_keep = principalDf['PC1'] < 0

to_keep.index = list(outlier_table.columns)
outlier_clean = outlier_table.loc[:,to_keep]

# Compute number of genes with log2 FC >= 1 b/w conditions and plot a histogram of the log2 FCs
cond1_means_corrected = ?
cond2_means_corrected = ?

outlier_corrected_fc = ?
DE_2 = ?

# make histogram
# here:

??

plt.title('With outlier detection') 
plt.xlabel('Log2 Fold Changes')
plt.ylabel('Density')
plt.show()

## Confounders

In [ ]:
# Load data and metadata for confounders
confounder_data = pd.read_csv('https://raw.githubusercontent.com/ds-connectors/Data88-Genetics_and_Genomics/master/Lab02/data_confounders.csv', sep = ' ', header = None)
confounder_meta = pd.read_csv('https://raw.githubusercontent.com/ds-connectors/Data88-Genetics_and_Genomics/master/Lab02/confounder_meta.csv', sep = ' ', header = None)

In [ ]:
# Estimate FCs between age groups ('Young' and 'Lld')
young_ind = confounder_meta.iloc[1,:] == 'Young'
old_ind = ?

young_mean = ?
old_mean = ?

age_fcs = ?

# make histogram
# here:

plt.title('Old vs Young') 
plt.xlabel('Log2 Fold Changes')
plt.ylabel('Density')
plt.show()

In [ ]:
# Compute FCs between age groups w/in each gender
ym_boolean = (confounder_meta.iloc[1,:] == 'Young') & (confounder_meta.iloc[0,:] == 'Male')
yf_boolean = ?
om_boolean = ?
of_boolean = ?

ym_mean = np.mean(confounder_data.loc[:, ym_boolean], axis = 1)
yf_mean = ?
om_mean = ?
of_mean = ?

# Old minus Young
male_age_fcs = ?
female_age_fcs = ?

In [ ]:
# Construct a 2x2 table of the metadata frequencies 
# (young/old vs male/female)

n_ym = np.sum(ym_boolean)
n_yf = ?
n_om = ?
n_of = ?

# Recall syntax to make a table
#new_table = Table().with_columns([
#    '', ['Colname1', 'Colname2'],
#    'Rowname1', [val11, val12],
#    'Rowname2', [val21, val22],
#])

metadata_table = ?

In [ ]:
# Load true age-related FCs (age_fc.txt)
age_fc_true = np.log2(pd.read_csv('https://raw.githubusercontent.com/ds-connectors/Data88-Genetics_and_Genomics/master/Lab02/age_fc.csv', sep = ' ', header = None))

In [ ]:
# Make plot of estiamted FCs for each cohort 
# (all, just men, just women) vs truth (put on same plot)

plt.scatter(?, ?, color = 'black')
plt.scatter(?, ?, color = 'blue')
plt.scatter(?, ?, color = 'red')

plt.title('Effect of Controlling for Confounders') 
plt.xlabel('True FCs')
plt.ylabel('Estimated FCs')
plt.legend(['Joint', 'Male', 'Female'])

plt.show()

# Putting it all together with real data

We'll now work with real RNA-seq samples primarily drawn from mammary tissue. However, there are also several samples which are actually blood and would negatively impact the analysis. Metadata is provided which tells whether a sample comes from a male/female and young/old donor. All data come from the GTEx consortium.

In [ ]:
# Load raw data from GTEx. Will take a few seconds
GTEx_data = pd.read_csv('https://raw.githubusercontent.com/ds-connectors/Data88-Genetics_and_Genomics/master/Lab02/GTEX_raw_mammary_blood.csv', sep = ' ')

1) Compute the sum of total reads per sample. Remember to not include the index column when calculating (the last column). Then answer the question, will we need to normalize?

In [ ]:
# Compute sum of reads
GTEx_TC = ?

# Make a barplot of the total number of reads
y_pos = np.arange(len(GTEx_TC))
plt.figure(figsize=(12,6))
plt.bar(? , ?)      # Fill in the missing places
plt.title('GTEx data without normalization') 
plt.xlabel('Sample')
plt.ylabel('Total reads')
plt.show()

2) Filter out lowly expressed genes (genes where the means across all patients is < 100 reads).

In [ ]:
# Filter out lowly expressed genes (mean across samples <= 100)

GTEx_means = ?
sufficient_reads = ?
GTEx_filtered = GTEx_data.loc[sufficient_reads,:]

3)  PCA to identify blood samples

In [ ]:
# PCA to identify outliers

# Run PCA and put the top PCs into a data frame
x = np.log2(GTEx_filtered+1)
x = StandardScaler().fit_transform(x.transpose())
pca = PCA(n_components=5)
principalComponents = pca.fit_transform(x)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['PC1', 'PC2', 'PC3', 'PC4', 'PC5'])

In [ ]:
# Plot results of PCA and look for outliers
plt.scatter(principalDf['PC1'], principalDf['PC2'])
plt.xlabel('PC 1')
plt.ylabel('PC 2')
plt.show()

4) Remove outliers

In [ ]:
# Drop outliers
to_keep = ?

to_keep.index = list(GTEx_filtered.columns)
GTEx_clean = GTEx_filtered.loc[:,to_keep]

5) Median normalization

In [ ]:
# Normalize data using median approach

medians = ?
GTEx_MN = ?

6) Load metadata

In [ ]:
# Load metadata and label features
GTEx_meta = pd.read_csv('https://raw.githubusercontent.com/ds-connectors/Data88-Genetics_and_Genomics/master/Lab02/GTEX_mammary_meta.csv', sep = ' ', header = None)
GTEx_meta.index = ['Gender', 'Age']

7) PCA, color points by gender

In [ ]:
# Run PCA and plot
x = np.log2(GTEx_MN+1)
x = StandardScaler().fit_transform(x.transpose())
pca = PCA(n_components=5)
principalComponents = pca.fit_transform(x)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['PC1', 'PC2', 'PC3', 'PC4', 'PC5'])

male_ind = (GTEx_meta.iloc[0,:] == 1)
female_ind = 

plt.scatter(principalDf['PC1'][male_ind], principalDf['PC2'][male_ind], color = 'Blue')
plt.scatter( ?, ?, color = 'Red')     # fill in missing entries
plt.xlabel('PC 1')
plt.ylabel('PC 2')
plt.legend(['Male', 'Female'])
plt.show()

8) Split np.log2(GTEx_MN+1) into tables for young and old women

In [ ]:
x = np.log2(GTEx_MN+1)

yw_ind = (GTEx_meta.iloc[0,:] == 2) & (GTEx_meta.iloc[1,:] <= 50)
ow_ind = ?

yw_ind.index = list(x.columns)
ow_ind.index = list(x.columns)

GTEx_yw = np.log2(GTEx_MN+1).loc[:,yw_ind]
GTEx_ow = ?

9) Compute FCs between old (>50) and young (<= 50) women and make a histogram

In [ ]:
age_fc_women = ?

# make a histogram
# here:

?

plt.xlabel('Log2 FC')
plt.ylabel('Density')
plt.title('Old vs young women, mammary tissue')
plt.show()

# To submit

In [ ]:
ok = Notebook('Lab02_Data_Cleaning.ok')
_ = ok.auth(inline=True)

In [ ]:
# Submit the assignment.
_ = ok.submit()